In [1]:
import pandas as pd
import numpy as np
from IPython.display import display
import random
from sklearn.model_selection import LeaveOneOut
from sklearn.linear_model import LinearRegression, LogisticRegression
import random
from sklearn.metrics import recall_score, confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split
from IPython.display import display

In [2]:
tagsFile = "tags"
recipesFile = "recipes"
nutritionsFile = "nutritions"
ingredientsFile = "ingredients"

tagsdf = pd.read_csv(f"/data/foodboost/{tagsFile}.csv", index_col=0)
recipesdf = pd.read_csv(f"/data/foodboost/{recipesFile}.csv", index_col=0)

In [3]:
def recepten_bij_tag(tag):
    a = tagsdf.loc[tagsdf['tag'] == tag].recipe.to_list()
    return a
def tags_bij_recept(gerecht):
    b = tagsdf.loc[tagsdf['recipe'] == gerecht].tag.unique()
    return b

list_of_tags = tagsdf['tag'].unique().tolist()

## Define Users

In [4]:
def User_Favo_Random_Tags(randomTag = random.choices(tagsdf["tag"].to_list(), k=1), K=10):
    #The Tag must have at least K dishes
    while len(recepten_bij_tag(randomTag[0])) < K:
        randomTag = random.choices(tagsdf["tag"].to_list(), k=1)
    
    #Defining tasty dishes
    RandomReceptenVoorTag = random.choices(recepten_bij_tag(randomTag[0]), k= K)
    
    #Split Dataset 80%-20%
    Train_Favorieten, Test_Favorieten = RandomReceptenVoorTag[:int(K*0.8)], RandomReceptenVoorTag[int(K*0.8):]
    
    #fill list with recipes for the User
    UserList_Tags = [tags_bij_recept(x) for x in Train_Favorieten]
    
    
    #Dislike (Or RANDOM THAT DO NOT HAVE TO DO WITH FAVOTAG)
    Niet_Favo_df = list(set(recipesdf["title"].to_list()).difference(recepten_bij_tag(randomTag[0])))
    Niet_Lekker_Recepten_Voor_Tag = random.choices(Niet_Favo_df, k= K)
    
    #Split Dataset 80%-20%
    Train_Favorieten_Niet_Lekker, Test_Favorieten_Niet_Lekker = Niet_Lekker_Recepten_Voor_Tag[:int(K*0.8)], \
																Niet_Lekker_Recepten_Voor_Tag[int(K*0.8):]
    
    #Save Tags for in the matrix
    Train_Favorieten_Niet_Lekker_Tags = [tags_bij_recept(x) for x in Train_Favorieten_Niet_Lekker]
    Test_Favorieten_Niet_Lekker_Tags = [tags_bij_recept(x) for x in Test_Favorieten_Niet_Lekker]

    return Train_Favorieten, UserList_Tags, Train_Favorieten_Niet_Lekker_Tags, Train_Favorieten_Niet_Lekker, \
			randomTag, K, Test_Favorieten, Test_Favorieten_Niet_Lekker_Tags

In [5]:
#Create Users
NumberOfUsers = 500
NumberOfRecipes = 20
UsersList = []
for i in range(NumberOfUsers):
    UserList, UserList_Tags, randomTags, randomRecipes, Usertag, K, Test_Favorieten, Test_Favorieten_Niet_Lekker_Tags \
		= User_Favo_Random_Tags(randomTag = random.choices(tagsdf["tag"].to_list(), k=1), K=NumberOfRecipes)
    
    User = np.array([UserList, UserList_Tags, randomTags, randomRecipes, Usertag, K, Test_Favorieten, \
						Test_Favorieten_Niet_Lekker_Tags])
    #Make list with all users
    UsersList.append(User)

ListSize = len(UsersList)
#Split dataset in Train- (60%), Validate- (20%) & Test (20%) set
TrainUsers, ValidateUsers, TestUsers = UsersList[:int(ListSize*0.6)], UsersList[int(ListSize*0.6): int(ListSize*0.8)], \
	UsersList[int(ListSize*0.8):]

/tmp/ipykernel_36296/2576423677.py:9: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  User = np.array([UserList, UserList_Tags, randomTags, randomRecipes, Usertag, K, Test_Favorieten, \


## Matrix Functions

In [6]:
def fillInMatrix(matrix, column, index_counter, doDoubleRows, isY, columnPrefix = ""):
    size = len(column)
    
    if(type(column) == str):
        size = 1
    
    #Iterate over the columns 
    for i in range(size):
        #Fill the current index and column with 1
        matrix.loc[index_counter, columnPrefix + column[i]] = 1
        
        #Check if we need to fill in double rows
        if(doDoubleRows):
            #If the column is 'y' fill the next row with 0, otherwise fill it with 1
            if(isY):
                matrix.loc[index_counter+1, columnPrefix + column[i]] = 0
            else:
                matrix.loc[index_counter+1, columnPrefix + column[i]] = 1

In [7]:
#TRAIN_Matrix
def generateTrainMatrix(UserList, randomTags, randomRecipes):
    #Create an empty dataframe with column names as the list of tags
    matrix = pd.DataFrame(columns = list_of_tags)
    
    #Create rows for all the users
    for i in range(len(UserList*2)):
        matrix.loc[matrix.shape[0]] = 0
    
    #Copy the matrix and add a prefix to column names
    columnPrefix = "2-"
    matrix2 = matrix.copy()
    matrix2.columns = [columnPrefix + columnName for columnName in matrix2.columns]
    
    X = np.array(UserList)
    loo = LeaveOneOut()
    index_counter = 0

    #Iterate over the train and test indices
    for train_index, test_index in loo.split(X):
        #Split the UserList into train and test sets
        X_train, X_test = X[train_index], X[test_index]

        #Pop the first element from the randomTags and randomRecipes lists
        randomTags0 = randomTags.pop(0)
        randomgerecht0 = randomRecipes.pop(0)

        #Get unique tags to fill the pivottable/pivotmatrix
        X_train_tags_unique = np.unique(np.concatenate([tags_bij_recept(x) for x in X_train]))
        X_test_tags_unique = np.unique(np.concatenate([tags_bij_recept(x) for x in X_test]))

        #----- Fill Trainmatrix
        fillInMatrix(matrix, X_train_tags_unique, index_counter, True, False)
        
        #Row 1
        fillInMatrix(matrix2, X_test_tags_unique, index_counter, False, False, columnPrefix = columnPrefix)
        #print(matrix2)
        
        #Row 2
        fillInMatrix(matrix2, randomTags0, index_counter+1, False, False, columnPrefix = columnPrefix)
        
        #Set the value for the random tag as 1 and put y at 0, because it should be false
        fillInMatrix(matrix2, 'y', index_counter, True, True)
        
        #Show which random tag is taken and the one out
        matrix2.loc[index_counter+1, 'Randomgerecht'] = str(randomgerecht0)
        matrix2.loc[index_counter, 'one out'] = X_test
        matrix2.loc[index_counter+1, 'one out'] = X_test
        
        index_counter += 2
    
    # Concatenate the two matrices and return the result
    return pd.concat([matrix, matrix2], axis=1)

In [8]:
#TEST_Matrix
def generateTestMatrix(UserList, Test_Favorieten, Test_Favorieten_Niet_Lekker_Tags):
    # Create an empty dataframe with column names as the list of tags
    matrix1 = pd.DataFrame(columns = list_of_tags)
    
    #Convert UserList and Test_Favorieten to numpy arrays
    Gerechten = np.array(UserList)
    Test_Gerechten = np.array(Test_Favorieten)
    
    #Create rows with all values set to 0
    for i in range(len(Test_Gerechten) + len(Test_Favorieten_Niet_Lekker_Tags)):
        matrix1.loc[matrix1.shape[0]] = 0
    
    # Copy the matrix and add a prefix to column names
    matrix2 = matrix1.copy()
    columnPrefix = "2-"
    matrix2.columns = [columnPrefix + columnName for columnName in matrix2.columns]
    
    # Iterate over the Test_Favorieten
    for index_counter in range(len(Test_Gerechten)):
        
        #Get unique tags for the UserList and the current Test_Favorieten
        Gerecht_Tags = np.unique(np.concatenate([tags_bij_recept(x) for x in Gerechten]))
        Test_Gerecht_Tags = tags_bij_recept(Test_Gerechten[index_counter])

        #----- Fill Matrix1 with Userlist (1-8)
        fillInMatrix(matrix1, Gerecht_Tags, index_counter, False, False)

        #----- Fill Matrix2 with Test_Favorieten (9-10)
        fillInMatrix(matrix2, Test_Gerecht_Tags, index_counter, False, False, columnPrefix = columnPrefix)
    
    
    # Iterate over the Test_Favorieten_Niet_Lekker_Tags
    for p in range(len(Test_Gerechten), len(Test_Gerechten) + len(Test_Favorieten_Niet_Lekker_Tags)):
        #----- Fill Matrix1 with Userlist (1-8)
        fillInMatrix(matrix1, Gerecht_Tags, p, False, False)
        
        fillInMatrix(matrix2, Test_Favorieten_Niet_Lekker_Tags, p, False, False, columnPrefix = columnPrefix)
    
    # Concatenate the two matrices and return the result
    return pd.concat([matrix1, matrix2], axis=1)

## Create Matrix

In [9]:
def createTrainMatrix():
    Matrix = pd.DataFrame()
    for User in TrainUsers:
        UserList, randomTags, randomRecipes = User[0], User[2], User[3]
        TrainMatrix = generateTrainMatrix(UserList, randomTags, randomRecipes)
        Matrix = pd.concat([Matrix, TrainMatrix], axis=0, ignore_index=True)
    
    return Matrix

In [10]:
def createValidateMatrix():
    Matrix = pd.DataFrame()
    y_validate = []
    for User in ValidateUsers:
        UserList, randomTags, randomRecipes, Test_Favorieten, Test_Favorieten_Niet_Lekker_Tags = \
			User[0], User[2], User[3], User[6], User[7]
        TestMatrix = generateTestMatrix(UserList, Test_Favorieten, Test_Favorieten_Niet_Lekker_Tags)
        Matrix = pd.concat([Matrix, TestMatrix], axis=0, ignore_index=True)
        
        #Create y_validate
        for p in Test_Favorieten:
            y_validate.append(1)
        for t in Test_Favorieten_Niet_Lekker_Tags:
            y_validate.append(0)
    return Matrix, y_validate

In [11]:
def createTestMatrix():
    Matrix = pd.DataFrame()
    y_test = []
    for User in TestUsers:
        UserList, randomTags, randomRecipes, Test_Favorieten, Test_Favorieten_Niet_Lekker_Tags = \
			User[0], User[2], User[3], User[6], User[7]
        TestMatrix = generateTestMatrix(UserList, Test_Favorieten, Test_Favorieten_Niet_Lekker_Tags)
        Matrix = pd.concat([Matrix, TestMatrix], axis=0, ignore_index=True)
        
        #Create y_test
        for p in Test_Favorieten:
            y_test.append(1)
        for t in Test_Favorieten_Niet_Lekker_Tags:
            y_test.append(0)
    return Matrix, y_test

In [12]:
TotalTrainMatrix = createTrainMatrix()
TotalValidateMatrix, y_validate = createValidateMatrix()
TotalTestMatrix, y_test = createTestMatrix()

In [13]:
TotalTrainMatrix

,hollands,gebak,sinterklaas,sinterklaasavond,oven,vooraf te maken,lactosevrij,thais,aziatisch,curry,...,2-valentijnsdag,2-zuid-afrikaans,2-koreaans,2-kidsfavoriet,2-diner,2-lente,2-jamie oliver,y,Randomgerecht,one out
0,1,0,0,0,0,1,1,0,1,0,...,0,0,0,0,0,0,0,1.0,NaN,[Pastinaaksoep met mosterd]
1,1,0,0,0,0,1,1,0,1,0,...,0,0,0,0,0,0,0,0.0,"Romige perline pasta met vegakip,pompoen &amp;...",[Pastinaaksoep met mosterd]
2,1,0,0,0,0,1,1,0,1,0,...,0,0,0,0,0,0,0,1.0,NaN,[Niçoise met peultjes en tuinerwten]
3,1,0,0,0,0,1,1,0,1,0,...,0,0,0,0,0,0,0,0.0,Woksaté met verse atjar,[Niçoise met peultjes en tuinerwten]
4,1,0,0,0,0,1,1,0,1,0,...,0,0,0,0,0,0,0,1.0,NaN,[Romige bietenrisotto met mortadella]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9595,1,1,1,1,1,1,1,0,0,0,...,0,0,0,0,0,0,0,0.0,Ravioli van mango en appel,[Appel-kaneelcupcakes]
9596,1,1,1,1,1,1,1,0,0,0,...,0,0,0,0,0,0,0,1.0,NaN,[Kokosyoghurttaart met mango]
9597,1,1,1,1,1,1,1,0,0,0,...,0,0,0,0,0,0,0,0.0,Knapperige flowersprouts,[Kokosyoghurttaart met mango]
9598,1,1,1,1,1,1,1,0,0,0,...,0,0,0,0,0,0,0,1.0,NaN,[Appeltaart van Rutger Bakt]


## Make train possible

In [14]:
X_train = TotalTrainMatrix.drop(['y', 'Randomgerecht', 'one out'], axis=1)
y_train = TotalTrainMatrix['y'].to_list()

In [15]:
X_train

,hollands,gebak,sinterklaas,sinterklaasavond,oven,vooraf te maken,lactosevrij,thais,aziatisch,curry,...,2-mealprep,2-slowcooker,2-oost-europees,2-valentijnsdag,2-zuid-afrikaans,2-koreaans,2-kidsfavoriet,2-diner,2-lente,2-jamie oliver
0,1,0,0,0,0,1,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,1,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,1,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,1,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,1,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9595,1,1,1,1,1,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9596,1,1,1,1,1,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9597,1,1,1,1,1,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9598,1,1,1,1,1,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
